In [1]:
import os
import sys
import glob
import sqlite3
from pathlib import Path
import ee
import geemap
import geopandas as gpd
import pandas as pd
import xee
import xarray as xr
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scs
import pyet
import spei

sys.path.append(r'C:\Users\Pooya\w\GitHub\ShiraziPooya\DroughtMonitoringIran')

from app.utils.gee import run_with_adaptive_buffer, extract_points_to_csv


ee.Authenticate()
ee.Initialize(
    project = 'drought-monitoring-iran',
    opt_url = 'https://earthengine-highvolume.googleapis.com'
)

In [2]:
DATABASE_PATH = "../database/database.db"

conn = sqlite3.connect(DATABASE_PATH)

gee_precip = pd.read_sql(sql='SELECT * FROM gee_precip_monthly', con=conn)
gee_precip['date'] = pd.to_datetime(gee_precip['date'])

geoinfo = pd.read_sql(sql='SELECT * FROM ground_data_geoinfo', con=conn)

conn.close()


# Download Data

In [ ]:
DATASETS = {
    "NDVI_Terra": {
        "image_collection_id": "MODIS/061/MOD13A3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "NDVI",
        "multiply": 0.0001,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "NDVI_Terra_MOD13A3",
        "output_path": f"../output/GEE/VI/NDVI_Terra_MOD13A3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    "EVI_Terra": {
        "image_collection_id": "MODIS/061/MOD13A3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "EVI",
        "multiply": 0.0001,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "EVI_Terra_MOD13A3",
        "output_path": f"../output/GEE/VI/EVI_Terra_MOD13A3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    "NDVI_Aqua": {
        "image_collection_id": "MODIS/061/MYD13A3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "NDVI",
        "multiply": 0.0001,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "NDVI_Aqua_MYD13A3",
        "output_path": f"../output/GEE/VI/NDVI_Aqua_MYD13A3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    "EVI_Aqua": {
        "image_collection_id": "MODIS/061/MYD13A3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "EVI",
        "multiply": 0.0001,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "EVI_Aqua_MYD13A3",
        "output_path": f"../output/GEE/VI/EVI_Aqua_MYD13A3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    "LST_Day_Terra": {
        "image_collection_id": "MODIS/061/MOD21C3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "LST_Day",
        "multiply": 0.02,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "LST_Day_Terra_MOD21C3",
        "output_path": f"../output/GEE/VI/LST_Day_Terra_MOD21C3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    "LST_Night_Terra": {
        "image_collection_id": "MODIS/061/MOD21C3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "LST_Night",
        "multiply": 0.02,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "LST_Night_Terra_MOD21C3",
        "output_path": f"../output/GEE/VI/LST_Night_Terra_MOD21C3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    "LST_Day_Aqua": {
        "image_collection_id": "MODIS/061/MYD21C3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "LST_Day",
        "multiply": 0.02,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "LST_Day_Aqua_MYD21C3",
        "output_path": f"../output/GEE/VI/LST_Day_Aqua_MYD21C3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    "LST_Night_Aqua": {
        "image_collection_id": "MODIS/061/MYD21C3",
        "start_date": "2000-01-01",
        "end_date": "2026-01-01",
        "parameter": "LST_Night",
        "multiply": 0.02,
        "add": 0.0,
        "scale": None,
        "unit": "number",
        "Cadence": "1 Month",
        "name": "LST_Night_Aqua_MYD21C3",
        "output_path": f"../output/GEE/VI/LST_Night_Aqua_MYD21C3.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
}

In [ ]:
for name, config in DATASETS.items():
    print(f"Product: {name}")
    df = run_with_adaptive_buffer(
        config=config,
        base_points_geojson=config["points_geojson"],
        buffer_list_m=[0, 1500, 2000, 2500, 5000, 7500, 10000],
    )  
    out_path = Path(config["output_path"])
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_path, index=False)
    print(f"Final merged result saved to {out_path}")

# Join *.CSV Files

In [3]:
folder = "../output/GEE/VI/"

results = pd.DataFrame()

for filepath in glob.glob(os.path.join(folder, "*.csv")):
    df = pd.read_csv(filepath)
    
    filename = os.path.basename(filepath)
    model_name, _ = os.path.splitext(filename)    
    model_name = model_name.split("-")[0]
    df = df[['region_id', 'Region', 'St_ID', 'St_Name', 'date', model_name]]

    
    if results.empty:
        results = df.copy()
    else:
        results = results.merge(df, on=['region_id', 'Region', 'St_ID', 'St_Name', 'date'], how='outer')
    
results.rename(
    columns={
        "Region": "region_name",
        "St_ID": "station_id",
        "St_Name": "station_name"
    },
    inplace=True
)

results = results[results["date"].apply(lambda x: x.split("-")[2] == "01")]
results['date'] = pd.to_datetime(results['date'])
results['date'] = results['date'] + pd.offsets.MonthEnd(0)

ndvi_columns = [col for col in results.columns if 'NDVI' in col]
results['NDVI'] = results[ndvi_columns].mean(axis=1)

evi_columns = [col for col in results.columns if 'EVI' in col]
results['EVI'] = results[evi_columns].mean(axis=1)

lst_day_columns = [col for col in results.columns if 'LST_Day' in col]
results['LST_Day'] = results[lst_day_columns].mean(axis=1)

lst_night_columns = [col for col in results.columns if 'LST_Night' in col]
results['LST_Night'] = results[lst_night_columns].mean(axis=1)

results['LST'] = results[['LST_Day', 'LST_Night']].mean(axis=1)

results = results.sort_values(by=["region_name", "region_id", "station_name", "station_id", "date"]).reset_index(drop=True)

results

,region_id,region_name,station_id,station_name,date,EVI_Aqua_MYD13A3,EVI_Terra_MOD13A3,LST_Day_Aqua_MYD21C3,LST_Day_Terra_MOD21C3,LST_Night_Aqua_MYD21C3,LST_Night_Terra_MOD21C3,NDVI_Aqua_MYD13A3,NDVI_Terra_MOD13A3,NDVI,EVI,LST_Day,LST_Night,LST
0,MASA,Mazandaran,99361,Alasht,2000-02-29,NaN,0.1194,NaN,5.4748,NaN,5.4344,NaN,0.2218,0.22180,0.11940,5.4748,5.4344,5.4546
1,MASA,Mazandaran,99361,Alasht,2000-03-31,NaN,0.1371,NaN,5.5564,NaN,5.5128,NaN,0.2558,0.25580,0.13710,5.5564,5.5128,5.5346
2,MASA,Mazandaran,99361,Alasht,2000-04-30,NaN,0.1942,NaN,5.9716,NaN,5.6512,NaN,0.3867,0.38670,0.19420,5.9716,5.6512,5.8114
3,MASA,Mazandaran,99361,Alasht,2000-05-31,NaN,0.2595,NaN,6.0724,NaN,5.6824,NaN,0.4515,0.45150,0.25950,6.0724,5.6824,5.8774
4,MASA,Mazandaran,99361,Alasht,2000-06-30,NaN,0.2217,NaN,6.1268,NaN,5.7572,NaN,0.3620,0.36200,0.22170,6.1268,5.7572,5.9420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4645,MASA,Mazandaran,40735,Siahbisheh,2025-07-31,0.3734,0.3853,6.2444,6.0232,5.7000,5.7144,0.5634,0.5523,0.55785,0.37935,6.1338,5.7072,5.9205
4646,MASA,Mazandaran,40735,Siahbisheh,2025-08-31,0.2985,0.2895,6.2796,6.0832,5.7392,5.7308,0.5440,0.5536,0.54880,0.29400,6.1814,5.7350,5.9582
4647,MASA,Mazandaran,40735,Siahbisheh,2025-09-30,0.1651,0.2486,6.1492,5.9760,5.6604,5.6540,0.4590,0.4897,0.47435,0.20685,6.0626,5.6572,5.8599
4648,MASA,Mazandaran,40735,Siahbisheh,2025-10-31,0.1323,0.1565,6.0180,5.8836,5.6256,5.5732,0.3729,0.3600,0.36645,0.14440,5.9508,5.5994,5.7751


# Calculate New Indices

## PCI

In [5]:
tmp_pci = gee_precip.copy()
tmp_pci['month'] = tmp_pci.date.dt.month

tmp_pci_stats = tmp_pci\
    .groupby(['region_id', 'station_id', 'month'])\
    .agg(
        ERA5min=('ERA5', 'min'),
        ERA5max=('ERA5', 'max'),
        
        GPMmin=('GPM', 'min'),
        GPMmax=('GPM', 'max'),
        
        TRMMmin=('TRMM', 'min'),
        TRMMmax=('TRMM', 'max'),
        
        TerraClimatemin=('TerraClimate', 'min'),
        TerraClimatemax=('TerraClimate', 'max'),
        
        PERSIANNCDRmin=('PERSIANN_CDR', 'min'),
        PERSIANNCDRmax=('PERSIANN_CDR', 'max'),
        
        CHIRPSmin=('CHIRPS', 'min'),
        CHIRPSmax=('CHIRPS', 'max')
    ).reset_index()

tmp_pci = tmp_pci.merge(tmp_pci_stats, on=['region_id', 'station_id', 'month'])
tmp_pci['PCI_ERA5'] = ((tmp_pci['ERA5'] - tmp_pci['ERA5min']) / (tmp_pci['ERA5max'] - tmp_pci['ERA5min']))
tmp_pci['PCI_GPM'] = ((tmp_pci['GPM'] - tmp_pci['GPMmin']) / (tmp_pci['GPMmax'] - tmp_pci['GPMmin']))
tmp_pci['PCI_TRMM'] = ((tmp_pci['TRMM'] - tmp_pci['TRMMmin']) / (tmp_pci['TRMMmax'] - tmp_pci['TRMMmin']))
tmp_pci['PCI_TerraClimate'] = ((tmp_pci['TerraClimate'] - tmp_pci['TerraClimatemin']) / (tmp_pci['TerraClimatemax'] - tmp_pci['TerraClimatemin']))
tmp_pci['PCI_PERSIANN_CDR'] = ((tmp_pci['PERSIANN_CDR'] - tmp_pci['PERSIANNCDRmin']) / (tmp_pci['PERSIANNCDRmax'] - tmp_pci['PERSIANNCDRmin']))
tmp_pci['PCI_CHIRPS'] = ((tmp_pci['CHIRPS'] - tmp_pci['CHIRPSmin']) / (tmp_pci['CHIRPSmax'] - tmp_pci['CHIRPSmin']))

tmp_pci.drop(
    columns=[
        'ERA5', 'GPM', 'TRMM', 'TerraClimate', 'PERSIANN_CDR', 'CHIRPS',
        'month', 'lat', 'lon', 'station_elevation',
        'ERA5min', 'ERA5max', 'GPMmin', 'GPMmax', 'TRMMmin', 'TRMMmax', 'TerraClimatemin', 'TerraClimatemax', 'PERSIANNCDRmin', 'PERSIANNCDRmax', 'CHIRPSmin', 'CHIRPSmax'
        ],
    inplace=True
)

tmp_pci['station_id'] = tmp_pci['station_id'].astype(int)

tmp_pci

,region_id,region_name,station_id,station_name,date,PCI_ERA5,PCI_GPM,PCI_TRMM,PCI_TerraClimate,PCI_PERSIANN_CDR,PCI_CHIRPS
0,MASA,Mazandaran,40732,Ramsar,1980-01-31,0.983334,NaN,NaN,0.736364,NaN,NaN
1,MASA,Mazandaran,40732,Ramsar,1980-02-29,0.563161,NaN,NaN,0.445161,NaN,NaN
2,MASA,Mazandaran,40732,Ramsar,1980-03-31,0.698348,NaN,NaN,0.761468,NaN,NaN
3,MASA,Mazandaran,40732,Ramsar,1980-04-30,0.337885,NaN,NaN,0.272727,NaN,NaN
4,MASA,Mazandaran,40732,Ramsar,1980-05-31,0.333260,NaN,NaN,0.200000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8245,MASA,Mazandaran,99361,Alasht,2025-06-30,0.370773,0.076923,NaN,NaN,NaN,0.115673
8246,MASA,Mazandaran,99361,Alasht,2025-07-31,0.000000,0.117021,NaN,NaN,NaN,0.006450
8247,MASA,Mazandaran,99361,Alasht,2025-08-31,0.103992,NaN,NaN,NaN,NaN,0.248408
8248,MASA,Mazandaran,99361,Alasht,2025-09-30,0.285254,NaN,NaN,NaN,NaN,0.050292


## VCI

In [6]:
tmp_vci = results.copy()[[
    'region_id', 'region_name', 'station_id', 'station_name', 'date',
    'NDVI'
]]

tmp_vci['month'] = tmp_vci.date.dt.month

tmp_vci_stats = tmp_vci.groupby(['region_id', 'region_name', 'station_id', 'station_name', 'month']).agg(
    NDVImin=('NDVI', 'min'),
    NDVImax=('NDVI', 'max'),
).reset_index()

tmp_vci = tmp_vci.merge(tmp_vci_stats, on=['region_id', 'region_name', 'station_id', 'station_name', 'month'])

tmp_vci['VCI'] = ((tmp_vci['NDVI'] - tmp_vci['NDVImin']) / (tmp_vci['NDVImax'] - tmp_vci['NDVImin']))

tmp_vci.drop(
    columns=[
        'month', 'NDVI',
        'NDVImin', 'NDVImax',

    ],
    inplace=True
)

tmp_vci

,region_id,region_name,station_id,station_name,date,VCI
0,MASA,Mazandaran,99361,Alasht,2000-02-29,0.424467
1,MASA,Mazandaran,99361,Alasht,2000-03-31,0.000000
2,MASA,Mazandaran,99361,Alasht,2000-04-30,0.247150
3,MASA,Mazandaran,99361,Alasht,2000-05-31,0.189469
4,MASA,Mazandaran,99361,Alasht,2000-06-30,0.000000
...,...,...,...,...,...,...
4645,MASA,Mazandaran,40735,Siahbisheh,2025-07-31,0.625000
4646,MASA,Mazandaran,40735,Siahbisheh,2025-08-31,0.882423
4647,MASA,Mazandaran,40735,Siahbisheh,2025-09-30,0.692975
4648,MASA,Mazandaran,40735,Siahbisheh,2025-10-31,0.586815


## TCI

In [7]:
tmp_tci = results.copy()[[
    'region_id', 'region_name', 'station_id', 'station_name', 'date',
    'LST', 'LST_Day', 'LST_Night'
]]

tmp_tci['month'] = tmp_tci.date.dt.month

tmp_tci_stats = tmp_tci.groupby(['region_id', 'region_name', 'station_id', 'station_name', 'month']).agg(
    LSTmin=('LST', 'min'),
    LSTmax=('LST', 'max'),
    LSTDaymin=('LST_Day', 'min'),
    LSTDaymax=('LST_Day', 'max'),
    LSTNightmin=('LST_Night', 'min'),
    LSTNightmax=('LST_Night', 'max')
).reset_index()

tmp_tci = tmp_tci.merge(tmp_tci_stats, on=['region_id', 'region_name', 'station_id', 'station_name', 'month'])

tmp_tci['TCI'] = ((tmp_tci['LSTmax'] - tmp_tci['LST']) / (tmp_tci['LSTmax'] - tmp_tci['LSTmin']))
tmp_tci['TCI_Day'] = ((tmp_tci['LSTDaymax'] - tmp_tci['LST_Day']) / (tmp_tci['LSTDaymax'] - tmp_tci['LSTDaymin']))
tmp_tci['TCI_Night'] = ((tmp_tci['LSTNightmax'] - tmp_tci['LST_Night']) / (tmp_tci['LSTNightmax'] - tmp_tci['LSTNightmin']))

tmp_tci.drop(
    columns=[
        'month', 
        'LST', 'LSTmin', 'LSTmax',
        'LST_Day', 'LSTDaymin', 'LSTDaymax',
        'LST_Night', 'LSTNightmin', 'LSTNightmax'
    ],
    inplace=True
)

tmp_tci


,region_id,region_name,station_id,station_name,date,TCI,TCI_Day,TCI_Night
0,MASA,Mazandaran,99361,Alasht,2000-02-29,0.968320,1.000000,0.760697
1,MASA,Mazandaran,99361,Alasht,2000-03-31,1.000000,1.000000,0.430605
2,MASA,Mazandaran,99361,Alasht,2000-04-30,0.000000,0.241021,0.000000
3,MASA,Mazandaran,99361,Alasht,2000-05-31,0.175558,0.128521,0.284345
4,MASA,Mazandaran,99361,Alasht,2000-06-30,0.310811,0.444096,0.115523
...,...,...,...,...,...,...,...,...
4645,MASA,Mazandaran,40735,Siahbisheh,2025-07-31,0.652653,0.602073,0.682836
4646,MASA,Mazandaran,40735,Siahbisheh,2025-08-31,0.227531,0.405199,0.000000
4647,MASA,Mazandaran,40735,Siahbisheh,2025-09-30,0.683453,0.769136,0.508929
4648,MASA,Mazandaran,40735,Siahbisheh,2025-10-31,0.348432,0.482068,0.104982


## VHI

In [8]:
alpha = 0.5

tmp_vhi = tmp_vci.merge(
    right=tmp_tci,
    on=['region_id', 'region_name', 'station_id', 'station_name', 'date'],
).assign(
    VHI=lambda x: alpha * x['VCI'] + (1 - alpha) * x['TCI'],
    VHI_Day=lambda x: alpha * x['VCI'] + (1 - alpha) * x['TCI_Day'],
    VHI_Night=lambda x: alpha * x['VCI'] + (1 - alpha) * x['TCI_Night']
).filter(
    items=["region_id", "region_name", "station_id", "station_name", "date", "VHI", "VHI_Day", "VHI_Night"]
)

tmp_vhi

,region_id,region_name,station_id,station_name,date,VHI,VHI_Day,VHI_Night
0,MASA,Mazandaran,99361,Alasht,2000-02-29,0.696394,0.712234,0.592582
1,MASA,Mazandaran,99361,Alasht,2000-03-31,0.500000,0.500000,0.215302
2,MASA,Mazandaran,99361,Alasht,2000-04-30,0.123575,0.244086,0.123575
3,MASA,Mazandaran,99361,Alasht,2000-05-31,0.182513,0.158995,0.236907
4,MASA,Mazandaran,99361,Alasht,2000-06-30,0.155405,0.222048,0.057762
...,...,...,...,...,...,...,...,...
4645,MASA,Mazandaran,40735,Siahbisheh,2025-07-31,0.638827,0.613536,0.653918
4646,MASA,Mazandaran,40735,Siahbisheh,2025-08-31,0.554977,0.643811,0.441212
4647,MASA,Mazandaran,40735,Siahbisheh,2025-09-30,0.688214,0.731055,0.600952
4648,MASA,Mazandaran,40735,Siahbisheh,2025-10-31,0.467623,0.534441,0.345898


## CI (Composite Index)

In [10]:
a = 1/3 # TCI
b = 1/3 # VCI
c = 1 - a - b # PCI

tmp_ci = tmp_vci.merge(
    right=tmp_tci,
    on=['region_id', 'region_name', 'station_id', 'station_name', "date"],
).merge(
    right=tmp_pci,
    on=['region_id', 'region_name', 'station_id', 'station_name', "date"],
).assign(
    CI_GPM=lambda x: a * x['TCI'] + b * x['VCI'] + c * x['PCI_GPM'],
).assign(
    CI_ERA5=lambda x: a * x['TCI'] + b * x['VCI'] + c * x['PCI_ERA5'],
).assign(
    CI_TRMM=lambda x: a * x['TCI'] + b * x['VCI'] + c * x['PCI_TRMM'],
).assign(
    CI_TerraClimate=lambda x: a * x['TCI'] + b * x['VCI'] + c * x['PCI_TerraClimate'],
).assign(
    CI_PERSIANN_CDR=lambda x: a * x['TCI'] + b * x['VCI'] + c * x['PCI_PERSIANN_CDR'],
).assign(
    CI_CHIRPS=lambda x: a * x['TCI'] + b * x['VCI'] + c * x['PCI_CHIRPS'],
).filter(
    items=['region_id', 'region_name', 'station_id', 'station_name', "date", "CI_GPM", "CI_ERA5", "CI_TRMM", "CI_TerraClimate", "CI_PERSIANN_CDR", "CI_CHIRPS"]
)

tmp_ci

,region_id,region_name,station_id,station_name,date,CI_GPM,CI_ERA5,CI_TRMM,CI_TerraClimate,CI_PERSIANN_CDR,CI_CHIRPS
0,MASA,Mazandaran,99361,Alasht,2000-02-29,0.620784,0.531348,0.692436,0.604613,0.514015,0.612052
1,MASA,Mazandaran,99361,Alasht,2000-03-31,0.405405,0.451744,0.407064,0.409910,0.418689,0.526284
2,MASA,Mazandaran,99361,Alasht,2000-04-30,0.082383,0.093177,0.082383,0.082383,0.082383,0.137346
3,MASA,Mazandaran,99361,Alasht,2000-05-31,0.155574,0.159546,0.141760,0.166873,0.128717,0.167123
4,MASA,Mazandaran,99361,Alasht,2000-06-30,0.261723,0.208007,0.277453,0.292009,0.178405,0.235777
...,...,...,...,...,...,...,...,...,...,...,...
4630,MASA,Mazandaran,40735,Siahbisheh,2025-06-30,0.518534,0.600980,NaN,NaN,NaN,0.553125
4631,MASA,Mazandaran,40735,Siahbisheh,2025-07-31,0.425884,0.611577,NaN,NaN,NaN,0.455641
4632,MASA,Mazandaran,40735,Siahbisheh,2025-08-31,NaN,0.418217,NaN,NaN,NaN,0.562936
4633,MASA,Mazandaran,40735,Siahbisheh,2025-09-30,NaN,0.601164,NaN,NaN,NaN,0.495228


## Combine

In [13]:
dataset = results.merge(
    right=tmp_pci,
    on=['region_id', 'region_name', 'station_id', 'station_name', 'date'],
    how='outer'    
).merge(
    right=tmp_vci,
    on=['region_id', 'region_name', 'station_id', 'station_name', 'date'],
    how='outer'
).merge(
    right=tmp_tci,
    on=['region_id', 'region_name', 'station_id', 'station_name', 'date'],
    how='outer'  
).merge(
    right=tmp_vhi,
    on=['region_id', 'region_name', 'station_id', 'station_name', 'date'],
    how='outer'    
).merge(
    right=tmp_ci,
    on=['region_id', 'region_name', 'station_id', 'station_name', 'date'],
    how='outer'    
)

dataset.sort_values(by=['region_id', 'region_name', 'station_id', 'station_name', 'date'], inplace=True)

dataset

,region_id,region_name,station_id,station_name,date,EVI_Aqua_MYD13A3,EVI_Terra_MOD13A3,LST_Day_Aqua_MYD21C3,LST_Day_Terra_MOD21C3,LST_Night_Aqua_MYD21C3,...,TCI_Night,VHI,VHI_Day,VHI_Night,CI_GPM,CI_ERA5,CI_TRMM,CI_TerraClimate,CI_PERSIANN_CDR,CI_CHIRPS
0,MASA,Mazandaran,40732,Ramsar,1980-01-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MASA,Mazandaran,40732,Ramsar,1980-02-29,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MASA,Mazandaran,40732,Ramsar,1980-03-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MASA,Mazandaran,40732,Ramsar,1980-04-30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MASA,Mazandaran,40732,Ramsar,1980-05-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8260,MASA,Mazandaran,99361,Alasht,2025-07-31,0.2864,0.2798,6.1844,6.0816,5.7124,...,0.428325,0.537304,0.533705,0.493979,0.39721,0.358202,NaN,NaN,NaN,0.360352
8261,MASA,Mazandaran,99361,Alasht,2025-08-31,0.2340,0.2272,6.1936,6.1124,5.7120,...,0.395382,0.472464,0.482118,0.503936,NaN,0.349640,NaN,NaN,NaN,0.397778
8262,MASA,Mazandaran,99361,Alasht,2025-09-30,0.1721,0.2019,5.9716,5.9908,5.6376,...,0.811203,0.893470,0.893470,0.799072,NaN,0.690732,NaN,NaN,NaN,0.612411
8263,MASA,Mazandaran,99361,Alasht,2025-10-31,0.1606,0.1854,6.0020,5.8640,5.5844,...,0.397287,0.495809,0.474763,0.449534,NaN,0.367154,NaN,NaN,NaN,0.431725


In [14]:
conn = sqlite3.connect(DATABASE_PATH)
dataset.to_sql(name='gee_indices_monthly', con=conn, if_exists='replace', index=False)
conn.commit()
conn.close()